## The below code is currently needed to make finitediff install on Kaggle:

In [ ]:
mypaths = !python -c "import sysconfig; print(sysconfig.get_paths()['purelib'])"
mypath = mypaths[0]
!pip install trash-cli
!trash $mypath/numpy*
!pip install --upgrade numpy
!pip install finitediff
import os
os.kill(os.getpid(), 9)

## After this step install HyDesign as usual:

In [2]:
!pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git

  Cloning https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git to /tmp/pip-req-build-bxd9g6j_
  Running command git clone --filter=blob:none --quiet https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git /tmp/pip-req-build-bxd9g6j_
  Resolved https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git to commit f16d61bb0dd1920bbce29e6385d2f4c321e204c9
  Preparing metadata (setup.py) ... done


## Import HyDesign:

In [3]:
from hydesign.hpp_assembly_P2X import hpp_model_P2X as hpp_model
from hydesign.Parallel_EGO import get_kwargs, EfficientGlobalOptimizationDriver
from smt.applications.mixed_integer import FLOAT, INT

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Detect no of CPUs to setup optimization alogrithm

In [4]:
import multiprocessing
n_cpu = int(multiprocessing.cpu_count())
n_doe = int(n_cpu * 4)
n_clusters = int(n_cpu / 2)

In [5]:
# Simple example to size wind and electrolyzer only with a single core to run test machines and colab

inputs = {
    'example': 4,
    'name': None,
    'longitude': None,
    'latitude': None,
    'altitude': None,
    'input_ts_fn': None,
    'sim_pars_fn': None,

    'opt_var': "NPV_over_CAPEX",
    'num_batteries': 1,
    'n_procs': n_cpu,
    'n_doe': n_doe,
    'n_clusters': n_clusters,
    'n_seed': 0,
    'max_iter': 10,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 3e4,
    'tol': 1e-6,
    'min_conv_iter': 2,
    'work_dir': './',
    'hpp_model': hpp_model,
    }

kwargs = get_kwargs(inputs)
kwargs['variables'] = {
    'clearance [m]':
        {'var_type':'design',
          'limits':[10, 60],
          'types':INT
          },
        # {'var_type':'fixed',
        #   'value': 35
        #   a},
     'sp [W/m2]':
        {'var_type':'design',
         'limits':[200, 360],
         'types':INT
         },
    'p_rated [MW]':
        {'var_type':'design',
          'limits':[1, 10],
          'types':INT
          },
        # {'var_type':'fixed',
        #  'value': 6
         # },
    'Nwt':
        {'var_type':'design',
          'limits':[0, 400],
          'types':INT
          },
        # {'var_type':'fixed',
        #   'value': 200
        #   },
    'wind_MW_per_km2 [MW/km2]':
        # {'var_type':'design',
        #   'limits':[5, 9],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 7
          },
    'solar_MW [MW]':
        # {'var_type':'design',
        #   'limits':[0, 400],
        #   'types':INT
        #   },
        {'var_type':'fixed',
          'value': 200
          },
    'surface_tilt [deg]':
        # {'var_type':'design',
        #   'limits':[0, 50],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 25
          },
    'surface_azimuth [deg]':
        # {'var_type':'design',
        #   'limits':[150, 210],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 180
          },
    'DC_AC_ratio':
        # {'var_type':'design',
        #   'limits':[1, 2.0],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value':1.0,
          },
    'b_P [MW]':
        # {'var_type':'design',
        #   'limits':[0, 100],
        #   'types':INT
        #   },
        {'var_type':'fixed',
          'value': 50
          },
    'b_E_h [h]':
        # {'var_type':'design',
        #   'limits':[1, 10],
        #   'types':INT
        #   },
        {'var_type':'fixed',
          'value': 6
          },
    'cost_of_battery_P_fluct_in_peak_price_ratio':
        # {'var_type':'design',
        #   'limits':[0, 20],
        #   'types':FLOAT
        #   },
        {'var_type':'fixed',
          'value': 10},
    'ptg_MW [MW]':
        {'var_type':'design',
          'limits':[1, 50],
          'types':INT
          },

    }

EGOD = EfficientGlobalOptimizationDriver(**kwargs)

Selected example site:
---------------------------------------------------
case                                         Europe
name                               France_good_wind
longitude                                 -0.864258
latitude                                  48.744116
altitude                                      302.0
input_ts_fn    Europe/input_ts_France_good_wind.csv
sim_pars_fn                     Europe/hpp_pars.yml
price_fn                   Europe/2030-EL_PRICE.csv
price_col                                      FR_R
Name: 4, dtype: object


In [6]:
EGOD.run()





Sizing a HPP plant at France_good_wind:

longitude = -0.864258
latitude = 48.744116
altitude = 302.0





KeyboardInterrupt: 

In [ ]:
EGOD.results